# New Section

In [ ]:
#Do not run
#Format of input?
#2D input array into keras
!pip install --target=$nb_path moabb
!pip3 install --target=$nb_path torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --target=$nb_path braindecode
#!pip install --target=$nb_path mne
!pip install --target=$nb_path -U https://api.github.com/repos/braindecode/braindecode/zipball/master

In [1]:

import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)


#4/1AX4XfWhv3cBxSin7WYEfMUa4vnyi5O4Cx-MYYR-hW-PeCAYZQBj7CH1b248

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

In [2]:
import io
from google.colab import files
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json


def integers(a, b):
         return list(range(a, b+1))

#config
OutputType = "Body Temp"
ReadyInput = "/content/drive/My Drive/raw_braindecode_bt"
min_delta = 60*1
delta_threshold = min_delta
n_times_input = 256*45
rawlist = integers(1,12)
mainpath = "/content/drive/My Drive/RawEEG_M5/"
#end config


sys.path.insert(0,nb_path)

import cv2
import scipy.signal 
from PIL import Image
from numpy import asarray
import matplotlib
import pathlib
import mne
from braindecode.datautil import create_from_X_y
from braindecode.datautil import save_concat_dataset 
from braindecode.datautil import load_concat_dataset 
from braindecode.datasets import BaseConcatDataset
from braindecode.datasets import BaseDataset
import random
from mne.preprocessing import ICA

/content/notebooks/braindecode/datautil/__init__.py:15: UserWarning: create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y
  warn('create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y')


In [5]:
filexist = False
#savefile = BaseConcatDataset([])
#savefile = load_concat_dataset(ReadyInput,preload=True)
#filexist = True
!rm -rf "/content/drive/My Drive/raw_braindecode_bt"
!mkdir "/content/drive/My Drive/raw_braindecode_bt"

for i in rawlist:
  order = str(i)+"/"
  subpath = os.path.join(mainpath,order)

  ################################################
  output_fname = OutputType+".xlsx"
  data_file = os.path.join(subpath, output_fname )
  dump = pd.ExcelFile(data_file)
  HeartRate = dump.parse(OutputType)

  OutputData = pd.DataFrame(HeartRate[OutputType])
  if (HeartRate["TimeStamp"].size < 1):
    continue
  #################################################
  obci_fname = 'raw.txt'
  data_file = os.path.join(subpath, obci_fname )

  channel_names = ['Fp1', 'Fp2', 'C3', 'C4', 'T5', 'T6', 'O1','O2']
  channel_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg']
  sfreq = 250  # in Hertz
  montage = 'standard_1020'

  info = mne.create_info(channel_names, sfreq, channel_types)

  n_channels = 8
  sampling_rate = 250
  #info = mne.create_info(n_channels, sampling_rate)
  print(info)




  counter = 0
  window_ = 5
  jumpstep_ = 3
  InputData = []
  OutputData = {}
  queue = {}
  sum_ = {}
  data_arr = np.empty((0, 8))
  time = 0
  timein = 0
  eeg_id = ["1","2","3","4","5","6","7","8"]
  dummy = np.empty((1, 8))
  id = np.zeros((340,1))
  inlen = 0

  sphere_not_define = True
  def createList(n):
      lst = []
      for i in range(n+1):
          lst.append(i)
      return(lst)
  newraw = np.empty((9, 10000000))
  newraw_l = 0
  with open( data_file, 'r' ) as in_f:
      header = True
      while header:
          line = next( in_f )
          if line[0] != '%':
              header = False
      dump = True
      for line in in_f:
          temp = np.array(line.strip( ' \n' ).split( ', ' )[1:23]).astype(np.float)
          newraw[0][newraw_l] = temp[0]
          newraw[1][newraw_l]  = temp[1]
          newraw[2][newraw_l]  = temp[2]
          newraw[3][newraw_l]  = temp[3]
          newraw[4][newraw_l]  = temp[4]
          newraw[5][newraw_l]  = temp[5]
          newraw[6][newraw_l]  = temp[6]
          newraw[7][newraw_l]  = temp[7]
          newraw[8][newraw_l]  = temp[21]
          newraw_l += 1

      newraw = mne.filter.filter_data(newraw[:,0:newraw_l], sfreq, 0.5, 70, picks=createList(7))
      newraw = mne.filter.notch_filter(newraw, sfreq, 50, picks=createList(7))
              
      time_arr = newraw[8]
      
      custom_raw = mne.io.RawArray(newraw[0:8,0:newraw_l], info)
      custom_raw.set_montage('standard_1020')

      #ica = ICA(n_components=8, max_iter='auto', random_state=123)
      #ica.fit(custom_raw)
      #ica.apply(custom_raw)

      if (sphere_not_define):
                sphere = mne.make_sphere_model('auto', 'auto', custom_raw.info)
                src = mne.setup_volume_source_space(sphere=sphere, exclude=30., pos=15.)
                forward = mne.make_forward_solution(custom_raw.info, trans=None, src=src, bem=sphere)
                sphere_not_define = False

      custom_raw.set_eeg_reference(ref_channels='REST',forward=forward)

      newraw = np.vstack((custom_raw._data,time_arr[np.newaxis,:]))

      del custom_raw

      for temp in range(newraw_l):
          #np.vstack((data_arr,np.array( line.strip( ' \n' ).split( ', ' )[1:9] )))
          #np.insert(X, 0, 6., axis=0)
          if (dump):
              dump = False
          else:
              #temp = np.array(line.strip( ' \n' ).split( ', ' )[1:23]).astype( np.float )
              time = newraw[8][temp] 
          if (time < HeartRate["TimeStamp"][timein]):
              continue
          if (time > (HeartRate["TimeStamp"][timein]+delta_threshold)):
              counter = 0
              timein += 1      
              data_arr = np.empty((0, 8))
              continue
          counter = counter + 1
          for x in range(8):
            dummy[:,x] = newraw[x][temp]
          data_arr = np.vstack((data_arr,dummy))
            
          if (counter == n_times_input):
              data_arr = data_arr.T
              

              target = HeartRate[OutputType][timein]

              datasets = []
              #save_concat_dataset("/content/drive/My Drive/raw_braindecode", windows_dataset)
              custom_raw = mne.io.RawArray(data_arr, info)

              #custom_raw.filter(0.5, 70)
              #custom_raw.notch_filter(60)
              custom_raw.set_montage('standard_1020')
              
              data_arr = np.empty((0, 8))

              status = random.randint(1, 10)

              if (status < 9 ):
                f_descrition = pd.Series(
                  data=[target, "train"],
                  index=["target", "session"])
              else:
                f_descrition = pd.Series(
                  data=[target, "test"],
                  index=["target", "session"])
              base_ds = BaseDataset(custom_raw, f_descrition, target_name="target")
              datasets.append(base_ds)
              InputData = BaseConcatDataset(datasets)
              if (filexist):
                savefile = BaseConcatDataset([savefile,InputData])
              else: 
                savefile = BaseConcatDataset(datasets) 
                filexist = True
              print(timein)
              timein += 1
              if (timein >= HeartRate[OutputType].size):
                break
              data_arr = np.empty((0, 8))
              counter = 0
              print(custom_raw.n_times)
save_concat_dataset(ReadyInput, savefile)

#apply Z score later in preprocessing before training



<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, Fp2, C3, C4, T5, T6, O1, O2
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 1651 samples (6.604 sec)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stop

<ipython-input-5-9b3c63abb38f>:96: RuntimeWarning: Only 8 head digitization points of the specified kinds ("eeg", "extra",), fitting may be inaccurate
  sphere = mne.make_sphere_model('auto', 'auto', custom_raw.info)



Finished.
EEG channel type selected for re-referencing
Applying REST reference.
Applying a custom ('EEG',) reference.
    8 out of 8 channels remain after picking
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
5
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
6
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
7
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
8
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
9
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
10
11520
Creating RawArray w

<ipython-input-5-9b3c63abb38f>:96: RuntimeWarning: Only 8 head digitization points of the specified kinds ("eeg", "extra",), fitting may be inaccurate
  sphere = mne.make_sphere_model('auto', 'auto', custom_raw.info)



Finished.
EEG channel type selected for re-referencing
Applying REST reference.
Applying a custom ('EEG',) reference.
    8 out of 8 channels remain after picking
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
5
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
6
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
7
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
8
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
9
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
10
11520
Creating RawArray w

...


Finished.
EEG channel type selected for re-referencing
Applying REST reference.
Applying a custom ('EEG',) reference.
    8 out of 8 channels remain after picking
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
5
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
6
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
7
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
8
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
9
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
10
11520
Creating RawArray w

<ipython-input-5-9b3c63abb38f>:96: RuntimeWarning: Only 8 head digitization points of the specified kinds ("eeg", "extra",), fitting may be inaccurate
  sphere = mne.make_sphere_model('auto', 'auto', custom_raw.info)



Finished.
EEG channel type selected for re-referencing
Applying REST reference.
Applying a custom ('EEG',) reference.
    8 out of 8 channels remain after picking
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
4
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
5
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
6
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
7
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
8
11520
Creating RawArray with float64 data, n_channels=8, n_times=11520
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
9
11520
Creating RawArray wi

/content/notebooks/braindecode/datautil/serialization.py:22: UserWarning: "save_concat_dataset()" is deprecated and will be removed in the future. Use dataset.save() instead.
  warnings.warn('"save_concat_dataset()" is deprecated and will be removed in'


Writing /content/drive/My Drive/raw_braindecode_bt/3/3-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/3/3-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/4/4-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/4/4-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/5/5-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/5/5-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/6/6-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/6/6-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/7/7-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/7/7-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/8/8-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/8/8-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/9/9-raw.fif
Closing /content/drive/My Drive/raw_braindecode_bt/9/9-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_bt/10/10-raw.fif
Clos

In [ ]:
!mkdir ReadyInput
save_concat_dataset(ReadyInput, savefile)

mkdir: cannot create directory ‘ReadyInput’: File exists


/content/notebooks/braindecode/datautil/serialization.py:22: UserWarning: "save_concat_dataset()" is deprecated and will be removed in the future. Use dataset.save() instead.
  warnings.warn('"save_concat_dataset()" is deprecated and will be removed in'


Writing /content/drive/My Drive/raw_braindecode_temp/0/0-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/0/0-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/1/1-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/1/1-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/2/2-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/2/2-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/3/3-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/3/3-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/4/4-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/4/4-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/5/5-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/5/5-raw.fif
[done]
Writing /content/drive/My Drive/raw_braindecode_temp/6/6-raw.fif
Closing /content/drive/My Drive/raw_braindecode_temp/6/6-raw.fif
[done]
Writing /content/drive/My Drive/raw_brain

In [12]:
from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.preprocess import zscore
from braindecode.models import Deep4Net
from braindecode.models import HybridNet
from braindecode.models import ShallowFBCSPNet
from braindecode import EEGRegressor
from braindecode.models.util import to_dense_prediction_model, get_output_shape
from braindecode.datautil import create_fixed_length_windows
import torch
from braindecode.training.losses import CroppedLoss
from braindecode.util import set_random_seeds, create_mne_dummy_raw
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

n_chans = 8
n_classes = 1
input_window_samples = n_times_input
n_epochs = 60
batch_size = 800

cuda = torch.cuda.is_available()

device = 'cuda' if cuda else 'cpu'

if cuda:
    torch.backends.cudnn.benchmark = True

if (True):
    model = Deep4Net(
        in_chans=n_chans,
        n_classes=n_classes,
        input_window_samples=input_window_samples,
        n_filters_time=25,
        n_filters_spat=25,
        stride_before_pool=True,
        n_filters_2=int(n_chans * 2),
        n_filters_3=int(n_chans * (2 ** 2.0)),
        n_filters_4=int(n_chans * (2 ** 3.0)),
        final_conv_length=1,
    )
    optimizer_lr = 0.3 #0.01
    optimizer_weight_decay = 0.0005
if (False):
    model = HybridNet(
        in_chans=n_chans,
        n_classes=n_classes,
        input_window_samples=input_window_samples,
    )
    optimizer_lr = 0.01
    optimizer_weight_decay = 0.0005

if (False):
    model = ShallowFBCSPNet(
        in_chans=n_chans,
        n_classes=n_classes,
        input_window_samples=input_window_samples,
        n_filters_time=40,
        n_filters_spat=40,
        final_conv_length=35,
    )
    optimizer_lr = 0.000625
    optimizer_weight_decay = 0

new_model = torch.nn.Sequential()
for name, module_ in model.named_children():
    if "softmax" in name:
        continue
    new_model.add_module(name, module_)
model = new_model

if cuda:
    model.cuda()

to_dense_prediction_model(model)
n_preds_per_input = get_output_shape(model, n_chans, n_times_input)[2]

savefile = load_concat_dataset(ReadyInput,preload=True)

windows_dataset = create_fixed_length_windows(
    savefile,
    start_offset_samples=0,
    stop_offset_samples=0,
    window_size_samples=input_window_samples,
    window_stride_samples=n_preds_per_input,
    drop_last_window=False,
    drop_bad_windows=True,
)

#preprocess(windows_dataset, [Preprocessor(zscore)])

splits = windows_dataset.split("session")
train_set = splits["train"]
valid_set = splits["test"]

regressor = EEGRegressor(
    model,
    cropped=True,
    criterion=CroppedLoss,
    criterion__loss_function=torch.nn.functional.mse_loss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),
    optimizer__lr=optimizer_lr,
    optimizer__weight_decay=optimizer_weight_decay,
    iterator_train__shuffle=True,
    batch_size=batch_size,
    callbacks=[
        "neg_root_mean_squared_error",
        # seems n_epochs -1 leads to desired behavior of lr=0 after end of training?
        ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
print(train_set.description["target"])
t = train_set.description["target"]
o = train_set.description["target"][0]

regressor.fit(train_set, y=train_set.description["target"], epochs=n_epochs)

/content/notebooks/braindecode/datautil/__init__.py:29: UserWarning: create_fixed_length_windows has been moved to preprocessing, please use from braindecode.preprocessing import create_fixed_length_windows
  warn(f'{name} has been moved to preprocessing, please use from braindecode.preprocessing import {name}')


Opening raw data file /content/drive/My Drive/raw_braindecode_bt/0/0-raw.fif...
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
Reading 0 ... 11519  =      0.000 ...    46.076 secs...
Opening raw data file /content/drive/My Drive/raw_braindecode_bt/1/1-raw.fif...
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
Reading 0 ... 11519  =      0.000 ...    46.076 secs...
Opening raw data file /content/drive/My Drive/raw_braindecode_bt/2/2-raw.fif...
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
Reading 0 ... 11519  =      0.000 ...    46.076 secs...
Opening raw data file /content/drive/My Drive/raw_braindecode_bt/3/3-raw.fif...
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
Reading 0 ... 11519  =      0.000 ...    46.076 secs...
Opening raw data file /content/drive/My Drive/raw_braindecode_bt/4/4-raw.fif...
    Range : 0 ... 11519 =      0.000 ...    46.076 secs
Ready.
Reading 0 ... 11519  =      0.000 ...    46.076 secs...
Openi

/content/notebooks/braindecode/preprocessing/windowers.py:597: UserWarning: Meaning of `trial_stop_offset_samples`=0 has changed, use `None` to indicate end of trial/recording. Using `None`.
  'Meaning of `trial_stop_offset_samples`=0 has changed, use `None` '


No baseline correction applied
0 projection items activated
Loading data for 1 events and 11520 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 11520 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 11520 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 11520 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1 matching events found
No baseline correction applied
0 project

In [11]:

predictions = regressor.predict(valid_set)
plt.scatter(valid_set.description["target"],predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
print(".")
plt.show()
print(regressor)
print(regressor.score(valid_set, valid_set.description["target"]))

Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 events and 11520 original time points ...
Loading data for 1 event

KeyboardInterrupt: ignored

In [7]:
 target = np.random.randint(0, 100, 11)
 print(target)
 target

[11  1 16 89 59 29 98 95 80 60 59]
